##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFRecord y tf.train.Example

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

El formato TFRecord es un formato simple para almacenar una secuencia de registros binarios.

Los [búferes de protocolo](https://developers.google.com/protocol-buffers/) son bibliotecas interlenguaje, interplataforma que se usan para la serialización eficiente de los datos estructurados.

Los archivos `.proto` definen los mensajes de protocolo. Por lo general, son la forma más fácil de entender un tipo de mensaje.

El mensaje `tf.train.Example` (o protobúfer) es un tipo de mensaje flexible que representa el mapeo de un `{"string": value}`. Está diseñado para ser usado con TensorFlow y se usa en las API de alto nivel como [TFX](https://www.tensorflow.org/tfx/).

En estas notas se demuestra cómo crear, analizar y usar el mensaje `tf.train.Example`. Después, se serializan, escriben y leen mensajes `tf.train.Example` hacia archivos `.tfrecord` y desde ellos.

Nota: Si bien son útiles, estas estructuras son opcionales. No hay necesidad de convertir códigos existentes para usar TFRecords, a menos que use [tf.data](https://www.tensorflow.org/guide/data) y lea datos, y todavía se genere un cuello de botella para el entrenamiento. Para más consejos sobre el rendimiento de los conjuntos de datos, consulte [Mejor rendimiento con la API tf.data](https://www.tensorflow.org/guide/data_performance).

Nota: En general, debería particionar horizontalmente los datos en múltiples archivos para poder paralelizar las entradas y salidas (dentro de un único <em>host</em> o en varios de ellos). Por regla general, habría que tener al menos 10 veces más archivos de los que los <em>hosts</em> leerán los datos. A la vez, cada archivo debería ser lo suficientemente grande (al menos más de 10 MB e idealmente más de 100 MB+) como para que la preextracción de entradas o salidas sea beneficiosa. Por ejemplo, digamos que tiene `X` GB de datos y que planea entrenar hasta `N` <em>hosts</em>. Lo ideal sería particionar horizontalmente los datos a archivos ~`10*N`, siempre y cuando ~`X/(10*N)` tenga más de 10 MB (y preferentemente más de 100 MB). Si el tamaño es menor, probablemente sea necesario generar menos particionamientos horizontales para compensar los beneficios del paralelismo y de la preextracción de entradas o salidas.

## Preparación

In [ ]:
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.train.Example`

### Tipos de datos para `tf.train.Example`

Fundamentalmente, un `tf.train.Example` es un mapeo `{"string": tf.train.Feature}`.

Para el tipo de mensaje `tf.train.Feature` se puede aceptar uno de los siguientes tipos (consulte el [archivo `.proto`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) para referencia). La mayoría de los demás tipos genéricos se pueden coercionar en uno de los siguientes:

1. `tf.train.BytesList` (los siguientes tipos se pueden coercionar)

- `string`
- `byte`

1. `tf.train.FloatList` (los siguientes tipos se pueden coercionar)

- `float` (`float32`)
- `double` (`float64`)

1. `tf.train.Int64List` (los siguientes tipos se pueden coercionar)

- `bool`
- `enum`
- `int32`
- `uint32`
- `int64`
- `uint64`

Para convertir a un tipo estándar de TensorFlow en un `tf.train.Example`, compatible con `tf.train.Feature`, puede usar las funciones de atajo que se encuentran debajo. Tenga en cuenta que cada función debe contar con un valor de entrada escalar y devuelve una `tf.train.Feature` que contiene uno de los tres tipos de `list` que se mostraron arriba:

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

Nota: Para no complejizar el tema, digamos que en este ejemplo solamente se usan entradas escalares. La manera más sencilla de manejar funciones no escalares es con `tf.io.serialize_tensor` para convertir tensores en cadenas binarias. Las cadenas son escalares en TensorFlow. Para volver a convertir la cadena binaria en un tensor, use `tf.io.parse_tensor`.

Más adelante hay algunos ejemplos sobre cómo trabajan estas funciones. Observe los diferentes tipos de entradas y los tipos estandarizados de salidas. Si el tipo de entrada de una función no coincide con uno de los tipos coercibles mencionados arriba, la función creará una excepción (p. ej., `_int64_feature(1.0)` mostrará un error porque `1.0` es flotante; por lo tanto, debería usarse con la función `_float_feature`):

In [ ]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

Todos los protomensajes se pueden serializar en una cadena binaria aplicando el método `.SerializeToString`:

In [ ]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### Creación de un mensaje `tf.train.Example`

Supongamos que desea crear un mensaje `tf.train.Example` a partir de datos existentes. En la práctica, el conjunto de datos puede provenir de cualquier parte, pero el procedimiento de crear el mensaje `tf.train.Example` a partir de una observación simple será siempre el mismo:

1. Dentro de cada observación, cada uno de los valores debe convertirse a `tf.train.Feature` y debe contener uno de los 3 tipos compatibles. Debe usar una de las funciones mencionadas arriba.

2. Cree un mapa (diccionario) que parta de la cadena del nombre de la función y llegue al valor de la función codificada producido en el número 1.

3. El mapa producido en el paso 2 se convierte en un [mensaje `Features`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85).

En este bloc de notas, creara un conjunto de datos con NumPy.

Este conjunto de datos tendrá 4 funciones:

- una función booleana, `False` o `True` con iguales probabilidades
- una función de enteros elegidos de manera aleatoria y uniforme a partir de `[0, 5]`
- una función de cadena generada a partir de una tabla de cadenas usando la función de enteros como índice.
- una función flotante de una distribución normal estándar

Piense en una muestra compuesta de 10 000 observaciones distribuidas de idéntico modo, para cada una de las distribuciones mencionadas arriba:

In [ ]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature.
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution.
feature3 = np.random.randn(n_observations)

Cada una de estas funciones se puede coercionar para generar un tipo `tf.train.Example` compatible usando una de las siguientes opciones: `_bytes_feature`, `_float_feature` o `_int64_feature`. Después, se puede crear un mensaje `tf.train.Example` a partir de estas funciones codificadas:

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

Por ejemplo, supongamos que tiene una observación simple de un conjunto de datos, `[False, 4, bytes('goat'), 0.9876]`. Con ello puede crear e imprimir el mensaje `tf.train.Example` para la observación con `create_message()`. Cada observación simple se escribirá como un mensaje `Features` según lo que se encuentra arriba. Tenga en cuenta que el [mensaje](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88) `tf.train.Example` solamente es un envoltorio (<em>wrapper</em>) en torno al mensaje `Features`:

In [ ]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

Para decodificar el mensaje, use el método `tf.train.Example.FromString`.

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

## Detalles del formato TFRecords

Un archivo TFRecord contiene una secuencia de registros. El archivo solamente se puede leer de forma secuencial.

Cada registro contiene una cadena de bytes, para una carga útil de datos, además de la longitud de los datos y CRC-32C ([CRC (verificación de redundancia cíclica) de 32-bit](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#CRC-32_algorithm) con los <em>hashes</em> del [polinomio de Castagnoli](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#Standards_and_common_use)) para control de integridad.

Los registros se almacenan en los siguientes formatos:

```
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

Los registros se concatenan juntos para producir el archivo. Las CRC [se describen aquí](https://en.wikipedia.org/wiki/Cyclic_redundancy_check) y la máscara de una CRC es la siguiente:

```
masked_crc = ((crc >> 15) | (crc << 17)) + 0xa282ead8ul
```


Nota: No es obligatorio usar `tf.train.Example` en archivos TFRecord. `tf.train.Example` solamente es un método que sirve para la serialización de diccionarios en cadenas de bytes. Cualquier cadena de bytes que se puede decodificar en TensorFlow se podría almacenar en un archivo TFRecord. Entre los ejemplos se encuentra lo siguiente: líneas de texto, JSON (con `tf.io.decode_json_example`), datos de imágenes codificados o `tf.Tensors` serializados (con `tf.io.serialize_tensor`/`tf.io.parse_tensor`). Para más opciones, consulte el módulo `tf.io`.

## Archivos TFRecord con `tf.data`

El módulo `tf.data` también brinda herramientas para leer y escribir datos en TensorFlow.

### Escritura de un archivo TFRecord

La forma más fácil de incluir los datos en un conjunto de datos es mediante la aplicación del método `from_tensor_slices`.

Aplicado a un arreglo, devuelve un conjunto de datos de escalares:

In [ ]:
tf.data.Dataset.from_tensor_slices(feature1)

Aplicado a un arreglo, devuelve un conjunto de datos de tuplas:

In [ ]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

In [ ]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)

Use el método `tf.data.Dataset.map` para aplicar la función en cada elemento de un `Dataset`.

La función mapeada debe trabajar en el modo de grafo de TensorFlow. Debe operar y devolver `tf.Tensors`. Una función sin tensores, como `serialize_example`, se puede encapsular en `tf.py_function` para hacerla compatible.

Para usar `tf.py_function` es necesario especificar el tamaño y el tipo de información que, de otro modo, no estaría disponible:

In [ ]:
def tf_serialize_example(f0,f1,f2,f3):
  tf_string = tf.py_function(
    serialize_example,
    (f0, f1, f2, f3),  # Pass these args to the above function.
    tf.string)      # The return type is `tf.string`.
  return tf.reshape(tf_string, ()) # The result is a scalar.

In [ ]:
tf_serialize_example(f0, f1, f2, f3)

Aplique esta función a cada elemento del conjunto de datos:

In [ ]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

In [ ]:
def generator():
  for features in features_dataset:
    yield serialize_example(*features)

In [ ]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [ ]:
serialized_features_dataset

Y escríbalos en un archivo TFRecord:

In [ ]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

### Lectura de un archivo TFRecord

También puede leer el archivo TFRecord con la clase `tf.data.TFRecordDataset`.

Para más información sobre archivos TFRecord de consumo que usan `tf.data`, consulte la guía [tf.data: compilar canalizaciones de entrada de TensorFlow](https://www.tensorflow.org/guide/data#consuming_tfrecord_data).

Los `TFRecordDataset` pueden ser útiles para estandarizar los datos de entrada y optimizar el rendimiento.

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

En este punto, el conjunto de datos contiene mensajes serializados `tf.train.Example`. Al iterar, los devuelve como tensores de cadenas escalares.

Use el método `.take` para mostrar solamente los primeros 10 registros.

Nota: Al iterar sobre un `tf.data.Dataset` solamente funciona la ejecución de tipo <em>eager</em> activa.

In [ ]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

Estos tensores se pueden analizar (<em>parse</em>) con la función que se encuentra debajo. Tenga en cuenta que aquí es necesario usar `feature_description` porque los `tf.data.Dataset` usan la ejecución basada en grafos y necesitan esta descripción para generar su propio tamaño y tipo de firma:

In [ ]:
# Create a description of the features.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

Como alternativa, use `tf.parse_example` para analizar el lote completo a la vez. Aplique esta función a cada elemento del conjunto de datos con el método `tf.data.Dataset.map`:

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

Use la ejecución <em>eager</em> para mostrar las observaciones en el conjunto de datos. En este conjunto de datos hay 10 000 observaciones, pero solamente se mostrarán las primeras 10. Los datos se muestran como un diccionario de características. Cada elemento es un `tf.Tensor` y el elemento `numpy` de ese tensor muestra el valor de la característica:

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

Aquí, la función `tf.parse_example` desempaqueta los campos `tf.train.Example` en tensores estándar.

## Archivos TFRecord en Python

El módulo `tf.io` también contiene funciones puras de Python para leer y escribir archivos de TFRecord.

### Escritura de un archivo TFRecord

A continuación, escriba las 10 000 observaciones en el archivo `test.tfrecord`. Cada observación se convierte en un mensaje `tf.train.Example`, que después se escribe en el archivo. Después, se puede verificar si el archivo `test.tfrecord` se ha creado:

In [ ]:
# Write the `tf.train.Example` observations to the file.
with tf.io.TFRecordWriter(filename) as writer:
  for i in range(n_observations):
    example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
    writer.write(example)

In [ ]:
!du -sh {filename}

### Lectura de un archivo TFRecord

Estos tensores serializados se pueden analizar fácilmente con `tf.train.Example.ParseFromString`:

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

Eso devuelve un prototipo `tf.train.Example` que resulta difícil de usar en el estado en que se encuentra, pero que, en esencia, es una representación de lo siguiente:

```
Dict[str,
     Union[List[float],
           List[int],
           List[str]]]
```

El siguiente código convierte manualmente el `Example` en un diccionario de arreglos NumPy, sin usar operaciones de TensorFlow Ops. Para más detalles, consulte el [archivo PROTO](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto).

In [ ]:
result = {}
# example.features.feature is the dictionary
for key, feature in example.features.feature.items():
  # The values are the Feature objects which contain a `kind` which contains:
  # one of three fields: bytes_list, float_list, int64_list

  kind = feature.WhichOneof('kind')
  result[key] = np.array(getattr(feature, kind).value)

result

## Paso a paso: lectura y escritura de datos de imágenes

Este es un ejemplo completo de cómo leer y escribir datos de imágenes con TFRecords. Con una imagen como dato de entrada, escribirá los datos como archivo TFRecord, después, vuelve a leer el archivo y muestra la imagen.

Puede resultar útil si, por ejemplo, se desea usar varios modelos en el mismo conjunto de datos de entrada. En vez de almacenar los datos sin procesar de la imagen, lo que puede hacer es procesarlos en formato TFRecords, y eso se puede usar en todos los siguientes procesamientos o modelados.

Primero, descargue [esta imagen](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg) de un gato en la nieve y [esta foto](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg) del puente de Williamsburg (Nueva York) en construcción.

### Extracción de las imágenes

In [ ]:
cat_in_snow  = tf.keras.utils.get_file(
    '320px-Felis_catus-cat_on_snow.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg')

williamsburg_bridge = tf.keras.utils.get_file(
    '194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [ ]:
display.display(display.Image(filename=cat_in_snow))
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
display.display(display.Image(filename=williamsburg_bridge))
display.display(display.HTML('<a "href=https://commons.wikimedia.org/wiki/File:New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg">From Wikimedia</a>'))

### Escritura del archivo TFRecord

Al igual que antes, codifique las características como tipos compatibles con `tf.train.Example`. De este modo, se almacena la característica de la cadena de la imagen sin procesar y también, la altura, el ancho, la profundidad y las características de la `label` arbitraria. Lo último se usa para escribir el archivo, para distinguir entre la imagen del gato y la del puente. Use `0` para la del gato y `1` para la del puente:

In [ ]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [ ]:
# This is an example, just using the cat image.
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

Note que todas las características ahora están almacenadas en el mensaje `tf.train.Example`. A continuación, funcionalice el código que figura arriba y escriba los mensajes del ejemplo en un archivo con el nombre `images.tfrecords`:

In [ ]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [ ]:
!du -sh {record_file}

### Lectura del archivo TFRecord

Ahora, tiene el archivo `images.tfrecords` y puede iterar sobre los registros que se encuentran en él para volver a leer lo que escribió inicialmente. Dado que en este ejemplo solamente se reproducirá la imagen, la única característica que necesitará será la cadena de la imagen sin procesar. Extráigala con los <em>getters</em> descriptos anteriormente, concretamente con `example.features.feature['image_raw'].bytes_list.value[0]`. También se pueden usar etiquetas para determinar qué registro es el del gato y cuál el del puente:

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Recupere las imágenes del archivo TFRecord:

In [ ]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))